In [1]:
from common import *
from structures import *

DAY = 24

In [2]:
show_task(DAY)

<article class="day-desc"><h2>--- Day 24: Never Tell Me The Odds ---</h2><p>It seems like something is going wrong with the snow-making process. Instead of forming snow, the water that's been absorbed into the air seems to be forming <a href="https://en.wikipedia.org/wiki/Hail" target="_blank">hail</a>!</p>
<p>Maybe there's something you can do to break up the hailstones?</p>
<p>Due to strong, probably-magical winds, the hailstones are all flying through the air in perfectly linear trajectories. You make a note of each hailstone's <em>position</em> and <em>velocity</em> (your puzzle input). For example:</p>
<pre><code>19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3
</code></pre>
<p>Each line of text corresponds to the position and velocity of a single hailstone. The positions indicate where the hailstones are <em>right now</em> (at time <code>0</code>). The velocities are constant and indicate exactly how far each hailstone will move in <em>one nanosecond</em>.</p>
<p>Each line of text uses the format <code>px py pz @ vx vy vz</code>. For instance, the hailstone specified by <code>20, 19, 15 @  1, -5, -3</code> has initial X position <code>20</code>, Y position <code>19</code>, Z position <code>15</code>, X velocity <code>1</code>, Y velocity <code>-5</code>, and Z velocity <code>-3</code>. After one nanosecond, the hailstone would be at <code>21, 14, 12</code>.</p>
<p>Perhaps you won't have to do anything. How likely are the hailstones to collide with each other and smash into tiny ice crystals?</p>
<p>To estimate this, consider only the X and Y axes; <em>ignore the Z axis</em>. Looking <em>forward in time</em>, how many of the hailstones' <em>paths</em> will intersect within a test area? (The hailstones themselves don't have to collide, just test for intersections between the paths they will trace.)</p>
<p>In this example, look for intersections that happen with an X and Y position each at least <code>7</code> and at most <code>27</code>; in your actual data, you'll need to check a much larger test area. Comparing all pairs of hailstones' future paths produces the following results:</p>
<pre><code>Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 18, 19, 22 @ -1, -1, -2
Hailstones' paths will cross <em>inside</em> the test area (at x=14.333, y=15.333).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths will cross <em>inside</em> the test area (at x=11.667, y=16.667).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=6.2, y=19.4).

Hailstone A: 19, 13, 30 @ -2, 1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone A.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 25, 34 @ -2, -2, -4
Hailstones' paths are parallel; they never intersect.

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-6, y=-5).

Hailstone A: 18, 19, 22 @ -1, -1, -2
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 12, 31, 28 @ -1, -2, -1
Hailstones' paths will cross outside the test area (at x=-2, y=3).

Hailstone A: 20, 25, 34 @ -2, -2, -4
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for hailstone B.

Hailstone A: 12, 31, 28 @ -1, -2, -1
Hailstone B: 20, 19, 15 @ 1, -5, -3
Hailstones' paths crossed in the past for both hailstones.
</code></pre>
<p>So, in this example, <code><em>2</em></code> hailstones' future paths cross inside the boundaries of the test area.</p>
<p>However, you'll need to search a much larger test area if you want to see if any hailstones might collide. Look for intersections that happen with an X and Y position each at least <code>200000000000000</code> and at most <code>400000000000000</code>. Disregard the Z axis entirely.</p>
<p>Considering only the X and Y axes, check all pairs of hailstones' future paths for intersections. <em>How many of these intersections occur within the test area?</em></p>
</article>

In [7]:
lines = '''19, 13, 30 @ -2,  1, -2
18, 19, 22 @ -1, -1, -2
20, 25, 34 @ -2, -2, -4
12, 31, 28 @ -1, -2, -1
20, 19, 15 @  1, -5, -3'''.splitlines()

In [ ]:
lines = get_input_lines(DAY)

In [10]:
rays = []
for line in lines:
    nums = [int(x) for x in line.replace(' @ ', ', ').split(',')]
    rays.append((nums[0], nums[1], nums[2], nums[3], nums[4], nums[5]))


In [23]:
from itertools import combinations

def intersect(ray1, ray2, min_pos: int = 7, max_pos: int = 27) -> bool:
    print('rays', ray1, ray2)
    x1, y1, z1, dx1, dy1, dz1 = ray1
    x2, y2, z2, dx2, dy2, dz2 = ray2
    if dx1 == dx2:
        if x1 != x2:
            print('  exit x1 != x2')
            return False
        xm = min_pos
    else:
        xm = (x2 - x1) / (dx1 - dx2)

    if dy1 == dy2:
        if y1 != y2:
            print('  exit y1 != y2')
            return False
        ym = min_pos
    else:
        ym = (y2 - y1) / (dy1 - dy2)

    print(f'  ok {xm=:.1f}, {ym=:.1f}')
    if min_pos <= xm <= max_pos and min_pos <= ym <= max_pos:
        return True
    return False

counter = 0
for ray1, ray2 in combinations(rays, 2):
    if intersect(ray1, ray2):
        counter += 1

print(counter)

rays (19, 13, 30, -2, 1, -2) (18, 19, 22, -1, -1, -2)
  ok xm=1.0, ym=3.0
rays (19, 13, 30, -2, 1, -2) (20, 25, 34, -2, -2, -4)
  exit x1 != x2
rays (19, 13, 30, -2, 1, -2) (12, 31, 28, -1, -2, -1)
  ok xm=7.0, ym=6.0
rays (19, 13, 30, -2, 1, -2) (20, 19, 15, 1, -5, -3)
  ok xm=-0.3, ym=1.0
rays (18, 19, 22, -1, -1, -2) (20, 25, 34, -2, -2, -4)
  ok xm=2.0, ym=6.0
rays (18, 19, 22, -1, -1, -2) (12, 31, 28, -1, -2, -1)
  exit x1 != x2
rays (18, 19, 22, -1, -1, -2) (20, 19, 15, 1, -5, -3)
  ok xm=-1.0, ym=0.0
rays (20, 25, 34, -2, -2, -4) (12, 31, 28, -1, -2, -1)
  exit y1 != y2
rays (20, 25, 34, -2, -2, -4) (20, 19, 15, 1, -5, -3)
  ok xm=-0.0, ym=-2.0
rays (12, 31, 28, -1, -2, -1) (20, 19, 15, 1, -5, -3)
  ok xm=-4.0, ym=-4.0
0


In [ ]:
send_result(DAY, 1, result)

In [ ]:
show_task(DAY, 2)

In [ ]:
lines = get_test_input_lines(DAY)

In [ ]:
lines = get_input_lines(DAY)

In [ ]:
send_result(DAY, 2, result)